# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import time
import json
import csv
import requests


# Define Functions

In [56]:
def sleep(x):
    time.sleep(x)

In [57]:
def matches(team_id, year) :
    url_matches = f"https://www.balldontlie.io/api/v1/games?seasons[]={year}&team_ids[]={team_id}"

    res_matches = requests.get(url_matches).json()
    total_pages = res_matches['meta']['total_pages']

    all_matches = []

    for page in range(1, total_pages + 1):
        url = f"{url_matches}&page={page}"
        res = requests.get(url).json()
        all_matches.extend(res['data'])

    df = pd.DataFrame(all_matches)
    return df

In [58]:
def stats(year) :
    url_stats = f"https://www.balldontlie.io/api/v1/stats?seasons[]={year}"

    res_stats = requests.get(url_stats).json()
    total_pages = 30
    #total_pages = res_stats['meta']['total_pages']

    all_stats = []

    for page in range(1, total_pages + 1):
        url = f"{url_stats}&page={page}"
        res = requests.get(url).json()
        all_stats.extend(res['data'])

    df = pd.DataFrame(all_stats)
    return df

## Get All teams ID's

In [91]:
all_teams = []
url_all_teams_1 = "https://www.balldontlie.io/api/v1/teams"
res_all_teams_1 = requests.get(url_all_teams_1).json()

url_all_teams_2 = "https://www.balldontlie.io/api/v1/teams?page=2"
res_all_teams_2 = requests.get(url_all_teams_2).json()

all_teams.extend(res_all_teams_1['data'])
all_teams.extend(res_all_teams_2['data'])
df_teams = pd.DataFrame(all_teams)
df_teams.head()

,id,abbreviation,city,conference,division,full_name,name
0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
1,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
2,3,BKN,Brooklyn,East,Atlantic,Brooklyn Nets,Nets
3,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
4,5,CHI,Chicago,East,Central,Chicago Bulls,Bulls


In [93]:
list01 = ['Phoenix Suns','Atlanta Hawks','LA Clippers','Milwaukee Bucks']
df_teams[['id', 'full_name']].loc[df_teams.full_name.isin(list01)]

,id,full_name
0,1,Atlanta Hawks
12,13,LA Clippers
16,17,Milwaukee Bucks
23,24,Phoenix Suns


In [94]:
df_teams.loc[df_teams.full_name.str.startswith("L")]

,id,abbreviation,city,conference,division,full_name,name
12,13,LAC,LA,West,Pacific,LA Clippers,Clippers
13,14,LAL,Los Angeles,West,Pacific,Los Angeles Lakers,Lakers


# Get Matches for Teams

In [59]:
df_1 = matches(1, 2021)
df_13 = matches(13, 2021)
df_17 = matches(17, 2021)
df_24 = matches(24, 2021)

In [60]:
df_1.head()

,id,date,home_team,home_team_score,period,postseason,season,status,time,visitor_team,visitor_team_score
0,473359,2021-10-04T00:00:00.000Z,"{'id': 16, 'abbreviation': 'MIA', 'city': 'Mia...",125,4,False,2021,Final,,"{'id': 1, 'abbreviation': 'ATL', 'city': 'Atla...",99
1,473424,2021-10-21T00:00:00.000Z,"{'id': 1, 'abbreviation': 'ATL', 'city': 'Atla...",113,4,False,2021,Final,,"{'id': 7, 'abbreviation': 'DAL', 'city': 'Dall...",87
2,473652,2021-11-20T00:00:00.000Z,"{'id': 1, 'abbreviation': 'ATL', 'city': 'Atla...",115,4,False,2021,Final,,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",105
3,473690,2021-11-26T00:00:00.000Z,"{'id': 15, 'abbreviation': 'MEM', 'city': 'Mem...",100,4,False,2021,Final,,"{'id': 1, 'abbreviation': 'ATL', 'city': 'Atla...",132
4,473685,2021-11-24T00:00:00.000Z,"{'id': 27, 'abbreviation': 'SAS', 'city': 'San...",106,4,False,2021,Final,,"{'id': 1, 'abbreviation': 'ATL', 'city': 'Atla...",124


In [96]:
# Game numbers for each team
list_df = [df_1 ,df_13, df_17, df_24]
[e.count()[0] for e in list_df]

[95, 90, 100, 101]

# Get Stats for matches ???

In [ ]:
'''
https://www.balldontlie.io/api/v1/stats
?seasons[]=2018&seasons[]=2015&player_ids[]=1&player_ids[]=2&postseason=true 
will return stats for player_ids 1 and 2 for the 2015-2016 and 2018-2019 postseason.
'''

In [105]:
df_stats = stats(2021)
df_stats.head()

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover
0,7065956,4,0,7,16.7,6,1,36.4,11,4,...,"{'id': 455103, 'date': '2021-08-10T00:00:00.00...",30:19,0,3,"{'id': 3547256, 'first_name': 'Saddiq', 'heigh...",12,7,0,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",3
1,7065957,1,0,3,0.0,0,0,75.0,4,3,...,"{'id': 455103, 'date': '2021-08-10T00:00:00.00...",13:48,3,1,"{'id': 666517, 'first_name': 'Tyler', 'height_...",10,6,0,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",1
2,7065958,2,1,3,0.0,3,0,37.5,8,3,...,"{'id': 455103, 'date': '2021-08-10T00:00:00.00...",24:28,2,4,"{'id': 666543, 'first_name': 'Sekou', 'height_...",7,5,1,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",1
3,7065959,2,1,4,44.4,9,4,44.4,18,8,...,"{'id': 455103, 'date': '2021-08-10T00:00:00.00...",29:07,0,7,"{'id': 17896075, 'first_name': 'Cade', 'height...",20,4,3,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",4
4,7065960,3,2,1,0.0,4,0,37.5,8,3,...,"{'id': 455103, 'date': '2021-08-10T00:00:00.00...",28:21,1,5,"{'id': 3547241, 'first_name': 'Killian', 'heig...",6,2,0,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",2


In [117]:
year = 2021
url_stats = f"https://www.balldontlie.io/api/v1/stats?seasons[]={year}&postseason=false"

res_stats = requests.get(url_stats).json()
total_pages = res_stats['meta']['total_pages']

all_stats = []

for page in range(1, total_pages + 1):
        url = f"{url_stats}&page={page}"
        res = requests.get(url).json()
        all_stats.extend(res['data'])
        print(page)
        sleep(0.75)


df = pd.DataFrame(all_stats)
df.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185


KeyboardInterrupt: 

In [181]:
df.to_csv("df_18020_year_2021_rest.csv", index=False)

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18020 entries, 0 to 18019
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        18020 non-null  int64  
 1   ast       18020 non-null  int64  
 2   blk       18020 non-null  int64  
 3   dreb      18020 non-null  int64  
 4   fg3_pct   16011 non-null  float64
 5   fg3a      18020 non-null  int64  
 6   fg3m      18020 non-null  int64  
 7   fg_pct    17951 non-null  float64
 8   fga       18020 non-null  int64  
 9   fgm       18020 non-null  int64  
 10  ft_pct    13604 non-null  float64
 11  fta       18020 non-null  int64  
 12  ftm       18020 non-null  int64  
 13  game      18020 non-null  object 
 14  min       18020 non-null  object 
 15  oreb      18020 non-null  int64  
 16  pf        18020 non-null  int64  
 17  player    18015 non-null  object 
 18  pts       18020 non-null  int64  
 19  reb       18020 non-null  int64  
 20  stl       18020 non-null  in

In [178]:
page

1376

In [179]:
total_pages

1376

In [149]:
res_stats

{'data': [{'id': 7065956,
   'ast': 4,
   'blk': 0,
   'dreb': 7,
   'fg3_pct': 16.7,
   'fg3a': 6,
   'fg3m': 1,
   'fg_pct': 36.4,
   'fga': 11,
   'fgm': 4,
   'ft_pct': 60.0,
   'fta': 5,
   'ftm': 3,
   'game': {'id': 455103,
    'date': '2021-08-10T00:00:00.000Z',
    'home_team_id': 11,
    'home_team_score': 111,
    'period': 0,
    'postseason': False,
    'season': 2021,
    'status': 'Final',
    'time': '',
    'visitor_team_id': 9,
    'visitor_team_score': 91},
   'min': '30:19',
   'oreb': 0,
   'pf': 3,
   'player': {'id': 3547256,
    'first_name': 'Saddiq',
    'height_feet': None,
    'height_inches': None,
    'last_name': 'Bey',
    'position': 'F',
    'team_id': 1,
    'weight_pounds': None},
   'pts': 12,
   'reb': 7,
   'stl': 0,
   'team': {'id': 9,
    'abbreviation': 'DET',
    'city': 'Detroit',
    'conference': 'East',
    'division': 'Central',
    'full_name': 'Detroit Pistons',
    'name': 'Pistons'},
   'turnover': 3},
  {'id': 7065957,
   'ast': 1,


In [169]:
len(all_stats)

16425

In [171]:
res_stats['meta']

{'total_pages': 1376,
 'current_page': 1,
 'next_page': 2,
 'per_page': 25,
 'total_count': 34395}

In [173]:
df_16425 = pd.DataFrame(all_stats)
df_16425.tail()


,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover
16420,9424458,6,2,3,0.25,8,2,0.412,17,7,...,"{'id': 474008, 'date': '2022-01-09T00:00:00.00...",34,0,3,"{'id': 3547238, 'first_name': 'Anthony', 'heig...",19,3,2,"{'id': 18, 'abbreviation': 'MIN', 'city': 'Min...",2
16421,9424464,5,0,5,0.00,1,0,0.714,7,5,...,"{'id': 474008, 'date': '2022-01-09T00:00:00.00...",18,0,1,"{'id': 666809, 'first_name': 'Jaylen', 'height...",11,5,3,"{'id': 18, 'abbreviation': 'MIN', 'city': 'Min...",0
16422,9424463,1,0,0,0.00,0,0,0.000,0,0,...,"{'id': 474008, 'date': '2022-01-09T00:00:00.00...",1:43,0,0,"{'id': 666767, 'first_name': 'Jordan', 'height...",0,0,0,"{'id': 18, 'abbreviation': 'MIN', 'city': 'Min...",0
16423,9424462,0,0,0,0.00,1,0,0.000,1,0,...,"{'id': 474008, 'date': '2022-01-09T00:00:00.00...",1:43,0,0,"{'id': 269, 'first_name': 'Jake', 'height_feet...",0,0,0,"{'id': 18, 'abbreviation': 'MIN', 'city': 'Min...",0
16424,9424461,0,0,1,NaN,0,0,NaN,0,0,...,"{'id': 474008, 'date': '2022-01-09T00:00:00.00...",2,0,1,"{'id': 3547295, 'first_name': 'Nathan', 'heigh...",2,1,0,"{'id': 18, 'abbreviation': 'MIN', 'city': 'Min...",1


In [174]:
df_16425.to_csv("df_16425_year_2021_half.csv", index=False)

In [146]:
df[df['game'].apply(lambda x: x.get('id') == 473424)].head(2)

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover


In [136]:
df[df['game'].apply(lambda x: x.get('date').startswith('2020'))]

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover


In [112]:
for i in df_stats['team']:
    print(i)

{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'Detroit Pistons', 'name': 'Pistons'}
{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'Detroit Pistons', 'name': 'Pistons'}
{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'Detroit Pistons', 'name': 'Pistons'}
{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'Detroit Pistons', 'name': 'Pistons'}
{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'Detroit Pistons', 'name': 'Pistons'}
{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'Detroit Pistons', 'name': 'Pistons'}
{'id': 9, 'abbreviation': 'DET', 'city': 'Detroit', 'conference': 'East', 'division': 'Central', 'full_name': 'D

In [185]:
df.head(1)

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover
0,9507392,0,0,0,0.0,0,0,0.0,0,0,...,"{'id': 474030, 'date': '2022-01-12T00:00:00.00...",0:00,0,0,"{'id': 3547282, 'first_name': 'Nick', 'height_...",0,0,0,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",0


In [186]:
df_16425.head(1)

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover
0,7065956,4,0,7,16.7,6,1,36.4,11,4,...,"{'id': 455103, 'date': '2021-08-10T00:00:00.00...",30:19,0,3,"{'id': 3547256, 'first_name': 'Saddiq', 'heigh...",12,7,0,"{'id': 9, 'abbreviation': 'DET', 'city': 'Detr...",3


In [187]:
pd.concat([df, df_16425]).drop_duplicates()

TypeError: unhashable type: 'dict'

In [194]:
df.select_dtypes(dict).columns

Index(['game', 'min', 'player', 'team'], dtype='object')

# Json Dosya Liste Halinde Okumak

In [40]:
path = "/Users/macbook/Desktop/IBM/DataScientest Serverdan Indirilen Dosyalar"
my_list = []
with open(f'{path}/stats_all.json', 'r') as file:
    for line in file:
        my_list.append(json.loads(line))

In [41]:
print("Type list_my = ", type(my_list))
print("Length List =",len(my_list))
my_list[-1]

Type list_my =  <class 'list'>
Length List = 1376


{'data': [{'id': 12292745,
   'ast': 2,
   'blk': 1,
   'dreb': 2,
   'fg3_pct': 0.0,
   'fg3a': 1,
   'fg3m': 0,
   'fg_pct': 0.33333334,
   'fga': 3,
   'fgm': 1,
   'ft_pct': 0.0,
   'fta': 2,
   'ftm': 0,
   'game': {'id': 473368,
    'date': '2021-10-05T00:00:00.000Z',
    'home_team_id': 5,
    'home_team_score': 131,
    'period': 4,
    'postseason': False,
    'season': 2021,
    'status': 'Final',
    'time': 'Final',
    'visitor_team_id': 6,
    'visitor_team_score': 95},
   'min': '22',
   'oreb': 0,
   'pf': 2,
   'player': {'id': 3547247,
    'first_name': 'Isaac',
    'height_feet': None,
    'height_inches': None,
    'last_name': 'Okoro',
    'position': 'F-G',
    'team_id': 6,
    'weight_pounds': None},
   'pts': 2,
   'reb': 2,
   'stl': 0,
   'team': {'id': 6,
    'abbreviation': 'CLE',
    'city': 'Cleveland',
    'conference': 'East',
    'division': 'Central',
    'full_name': 'Cleveland Cavaliers',
    'name': 'Cavaliers'},
   'turnover': 1},
  {'id': 1229274

In [43]:
df = pd.DataFrame(my_list)
df.tail()

,data,meta
1371,"[{'id': 11644700, 'ast': 2, 'blk': 0, 'dreb': ...","{'total_pages': 1376, 'current_page': 1372, 'n..."
1372,"[{'id': 11640866, 'ast': 0, 'blk': 0, 'dreb': ...","{'total_pages': 1376, 'current_page': 1373, 'n..."
1373,"[{'id': 12292521, 'ast': 7, 'blk': 1, 'dreb': ...","{'total_pages': 1376, 'current_page': 1374, 'n..."
1374,"[{'id': 12292546, 'ast': 1, 'blk': 0, 'dreb': ...","{'total_pages': 1376, 'current_page': 1375, 'n..."
1375,"[{'id': 12292745, 'ast': 2, 'blk': 1, 'dreb': ...","{'total_pages': 1376, 'current_page': 1376, 'n..."


# Json Normalize 

In [51]:
pd.json_normalize(my_list).head(1)

,data,meta.total_pages,meta.current_page,meta.next_page,meta.per_page,meta.total_count
0,"[{'id': 7065956, 'ast': 4, 'blk': 0, 'dreb': 7...",1376,1,2.0,25,34395


In [31]:
dfdata = pd.json_normalize(my_list, 'data')
dfdata.tail()

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name,player
34390,12292760,0,0,0,0.0,0,0,0.0,0,0,...,16.0,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN
34391,12292761,0,0,0,0.0,0,0,0.0,0,0,...,2.0,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN
34392,12292762,0,0,0,0.0,0,0,0.0,0,0,...,6.0,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN
34393,12292763,0,0,0,0.0,0,0,0.0,0,0,...,2.0,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN
34394,12292764,0,0,0,0.0,0,0,0.0,0,0,...,6.0,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN


In [55]:
dfmeta = pd.json_normalize(my_list)
dfmeta = dfmeta.drop(columns='data').tail()
dfmeta.tail()

,meta.total_pages,meta.current_page,meta.next_page,meta.per_page,meta.total_count
1371,1376,1372,1373.0,25,34395
1372,1376,1373,1374.0,25,34395
1373,1376,1374,1375.0,25,34395
1374,1376,1375,1376.0,25,34395
1375,1376,1376,NaN,25,34395


In [65]:
list_id01 = df_1['id'].to_list()
list_id01[:5]

[473359, 473424, 473652, 473690, 473685]

In [94]:
list_id01 = df_1['id'].to_list()
list_id13 = df_13['id'].to_list()
list_id17 = df_17['id'].to_list()
list_id24 = df_24['id'].to_list()
print(len(list_id01))

[list_id01.append(i) for i in list_id13]
[list_id01.append(i) for i in list_id17]
[list_id01.append(i) for i in list_id24]
print(len(list_id01))
print(len(set(list_id01)))

95
386
370


# Lsit Comp. in df columns

In [92]:
#df['C'] = [x[0] in x[1] for x in zip(df['A'], df['B'])]

list_tf1 = []
for i in list_id01:
    for x in dfdata['game.id']:
        if i == x:
            dfdata['tf'] = 1
            list_tf1.append(i)
            if len(list_tf1) % 25 == 0:
                print(len(list_tf1))
        else:
            dfdata['tf'] = 0

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
5000
5025
5050
5075
5100
5125
5150
5175
5200
52

In [99]:
len(set(list_tf1))

317

In [105]:
print(list_tf1[-20:])

[467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898, 467898]


In [109]:
dfdata[['id', 'team.id', 'game.home_team_id', 'game.home_team_id','game.id', 'tf']].loc[(dfdata['game.home_team_id'].isin([1, 13, 17, 24]))]

,id,team.id,game.home_team_id,game.home_team_id,game.id,tf
602,7116235,1,1,1,459132,0
603,7116215,23,1,1,459132,0
604,7116216,23,1,1,459132,0
605,7116217,23,1,1,459132,0
606,7116218,23,1,1,459132,0
...,...,...,...,...,...,...
34209,11398988,17,17,17,474561,0
34210,11398987,17,17,17,474561,0
34211,11398986,17,17,17,474561,0
34212,11398989,17,17,17,474561,0


In [97]:
dfdata.loc[dfdata['tf'] == 0]

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name,player,tf
0,7065956,4,0,7,16.7,6,1,36.4,11,4,...,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,NaN,0
1,7065957,1,0,3,0.0,0,0,75.0,4,3,...,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,NaN,0
2,7065958,2,1,3,0.0,3,0,37.5,8,3,...,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,NaN,0
3,7065959,2,1,4,44.4,9,4,44.4,18,8,...,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,NaN,0
4,7065960,3,2,1,0.0,4,0,37.5,8,3,...,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34390,12292760,0,0,0,0.0,0,0,0.0,0,0,...,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN,0
34391,12292761,0,0,0,0.0,0,0,0.0,0,0,...,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN,0
34392,12292762,0,0,0,0.0,0,0,0.0,0,0,...,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN,0
34393,12292763,0,0,0,0.0,0,0,0.0,0,0,...,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,NaN,0


In [75]:
dfdata.iloc[0]

id                                          7065956
ast                                               4
blk                                               0
dreb                                              7
fg3_pct                                        16.7
fg3a                                              6
fg3m                                              1
fg_pct                                         36.4
fga                                              11
fgm                                               4
ft_pct                                         60.0
fta                                               5
ftm                                               3
min                                           30:19
oreb                                              0
pf                                                3
pts                                              12
reb                                               7
stl                                               0
turnover    

In [76]:
dfdata['game.id'].head()

0    455103
1    455103
2    455103
3    455103
4    455103
Name: game.id, dtype: int64

In [79]:
df_1['id'].head()

0    473359
1    473424
2    473652
3    473690
4    473685
Name: id, dtype: int64

In [81]:
type(list_id01[0])

int